In [1]:
import tensorflow as tf
import os
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [2]:
data_dir = '/local-scratch/localhome/mkhademi/BOLD5000_2.0/'
batch_size = 20
dataset = tf.data.TFRecordDataset(filenames = [data_dir + 'image_data/bold5000_coco.tfrecords']).batch(batch_size)
train_ds = dataset.take(1800//batch_size)
test_ds = dataset.skip(1800//batch_size)
test_ds = test_ds.take(200//batch_size)
train_ds = train_ds.shuffle(1800)

In [3]:
def tf_parse(eg):
    example = tf.io.parse_example(
        eg, {
            'x': tf.io.FixedLenFeature(shape=(71, 89, 72), dtype=tf.float32),
            'y': tf.io.FixedLenFeature(shape=(1000), dtype=tf.float32),
            'y_coco': tf.io.FixedLenFeature(shape=(90), dtype=tf.int64),
            'y_imagenet': tf.io.FixedLenFeature(shape=(1000), dtype=tf.int64),
            'y_super': tf.io.FixedLenFeature(shape=(12), dtype=tf.int64),
            'y_common': tf.io.FixedLenFeature(shape=(90+12), dtype=tf.int64),
            'y_face': tf.io.FixedLenFeature(shape=(2), dtype=tf.int64),
        })
    return example['x'], example['y_face']
    
decoded = train_ds.map(tf_parse)
decoded_test = test_ds.map(tf_parse)

In [4]:
raw_example = next(iter(test_ds))
print(tf_parse(raw_example)[0].numpy().shape)
print(tf_parse(raw_example)[1].numpy().shape)
print(tf_parse(raw_example)[1].numpy().sum(axis=1))

(20, 71, 89, 72)
(20, 2)
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [5]:
for x_batch, y_batch in decoded_test.take(1):
    print(x_batch, y_batch)

tf.Tensor(
[[[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  ...

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0.

In [6]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(71, 89, 72)),
    tf.keras.layers.Conv2D(4, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(8, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(16, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(rate=0.5),
    # tf.keras.layers.Dense(1000, activation='sigmoid'),
    # tf.keras.layers.Dense(100, activation='sigmoid'),
    tf.keras.layers.Dense(2),
    # tf.keras.layers.Softmax()
    ])
 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 69, 87, 4)         2596      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 34, 43, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 41, 8)         296       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 20, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 18, 16)        1168      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 9, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1008)              0

In [7]:
model.compile(
    # loss='mse',
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    # loss = tf.keras.losses.BinaryCrossentropy(from_logits=False),
    # optimizer=keras.optimizers.RMSprop(),
    optimizer='adam',
    # metrics=[tf.keras.metrics.MeanSquaredError()],
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
    # metrics=[tf.keras.metrics.CategoricalCrossentropy(from_logits=True)]
)
model.fit(decoded, epochs=60)

Epoch 1/60
90/90 [==============================] - 6s 17ms/step - loss: 121.0283 - categorical_accuracy: 0.6139
Epoch 2/60
90/90 [==============================] - 4s 17ms/step - loss: 50.8925 - categorical_accuracy: 0.6344
Epoch 3/60
90/90 [==============================] - 4s 16ms/step - loss: 62.5480 - categorical_accuracy: 0.6456
Epoch 4/60
90/90 [==============================] - 4s 16ms/step - loss: 35.0018 - categorical_accuracy: 0.6328
Epoch 5/60
90/90 [==============================] - 4s 16ms/step - loss: 21.8706 - categorical_accuracy: 0.6456
Epoch 6/60
90/90 [==============================] - 4s 16ms/step - loss: 17.7047 - categorical_accuracy: 0.6556
Epoch 7/60
90/90 [==============================] - 4s 16ms/step - loss: 14.7418 - categorical_accuracy: 0.6567
Epoch 8/60
90/90 [==============================] - 4s 17ms/step - loss: 8.1556 - categorical_accuracy: 0.6667
Epoch 9/60
90/90 [==============================] - 4s 16ms/step - loss: 5.9541 - categorical_accuracy: 

In [8]:
loss, accuracy = model.evaluate(decoded_test)

10/10 [==============================] - 2s 24ms/step - loss: 0.9684 - categorical_accuracy: 0.6950


In [9]:
model.add(tf.keras.layers.Softmax())
for x_batch, y_batch in decoded_test.take(1):
    print(model(x_batch))

tf.Tensor(
[[8.6732537e-01 1.3267466e-01]
 [2.5921864e-02 9.7407812e-01]
 [3.3589083e-01 6.6410917e-01]
 [2.5587147e-01 7.4412847e-01]
 [1.2557174e-04 9.9987435e-01]
 [1.8507537e-01 8.1492466e-01]
 [7.7341890e-01 2.2658114e-01]
 [3.6768239e-02 9.6323174e-01]
 [4.2119464e-01 5.7880533e-01]
 [1.0280206e-05 9.9998975e-01]
 [3.9054159e-02 9.6094579e-01]
 [2.1277349e-01 7.8722656e-01]
 [9.0286784e-02 9.0971315e-01]
 [5.1210675e-04 9.9948788e-01]
 [6.1172905e-05 9.9993885e-01]
 [1.4774777e-01 8.5225224e-01]
 [7.6067368e-03 9.9239326e-01]
 [2.3947825e-01 7.6052171e-01]
 [6.3254531e-03 9.9367452e-01]
 [3.7950773e-02 9.6204919e-01]], shape=(20, 2), dtype=float32)


In [10]:
batch_size = 20
dataset = tf.data.TFRecordDataset(filenames = [data_dir + 'image_data/bold5000_common.tfrecords']).batch(batch_size)
train_ds = dataset.take((2135+400)//batch_size)
test_ds = dataset.skip((2135+400)//batch_size)
test_ds = test_ds.take(329//batch_size)
train_ds = train_ds.shuffle(2135+400)

In [11]:
def tf_parse(eg):
    example = tf.io.parse_example(
        eg, {
            'x': tf.io.FixedLenFeature(shape=(71, 89, 72), dtype=tf.float32),
            'y': tf.io.FixedLenFeature(shape=(1000), dtype=tf.float32),
            'y_coco': tf.io.FixedLenFeature(shape=(90), dtype=tf.int64),
            'y_imagenet': tf.io.FixedLenFeature(shape=(1000), dtype=tf.int64),
            'y_super': tf.io.FixedLenFeature(shape=(12), dtype=tf.int64),
            'y_common': tf.io.FixedLenFeature(shape=(90+12), dtype=tf.int64),
            'y_face': tf.io.FixedLenFeature(shape=(2), dtype=tf.int64),
        })
    return example['x'], example['y_super']
    
decoded = train_ds.map(tf_parse)
decoded_test = test_ds.map(tf_parse)

In [12]:
raw_example = next(iter(test_ds))
print(tf_parse(raw_example)[0].numpy().shape)
print(tf_parse(raw_example)[1].numpy().shape)
print(tf_parse(raw_example)[1].numpy().sum(axis=1))

(20, 71, 89, 72)
(20, 12)
[1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1]


In [13]:
for x_batch, y_batch in decoded_test.take(1):
    print(x_batch, y_batch)

tf.Tensor(
[[[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  ...

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0.

In [14]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(71, 89, 72)),
    tf.keras.layers.Conv2D(4, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(8, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(16, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(rate=0.5),
    # tf.keras.layers.Dense(1000, activation='sigmoid'),
    # tf.keras.layers.Dense(100, activation='sigmoid'),
    tf.keras.layers.Dense(12),
    # tf.keras.layers.Softmax()
    ])
 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 69, 87, 4)         2596      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 34, 43, 4)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 41, 8)         296       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 20, 8)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 18, 16)        1168      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 9, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1008)             

In [15]:
model.compile(
    # loss='mse',
    # loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
    # optimizer=keras.optimizers.RMSprop(),
    optimizer='adam',
    # metrics=[tf.keras.metrics.MeanSquaredError()],
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
    # metrics=[tf.keras.metrics.CategoricalCrossentropy(from_logits=True)]
)
model.fit(decoded, epochs=60)

Epoch 1/60
126/126 [==============================] - 6s 16ms/step - loss: 28.5778 - categorical_accuracy: 0.1504
Epoch 2/60
126/126 [==============================] - 5s 17ms/step - loss: 6.9820 - categorical_accuracy: 0.1353
Epoch 3/60
126/126 [==============================] - 5s 16ms/step - loss: 3.2951 - categorical_accuracy: 0.1349
Epoch 4/60
126/126 [==============================] - 5s 16ms/step - loss: 1.6545 - categorical_accuracy: 0.1627
Epoch 5/60
126/126 [==============================] - 5s 17ms/step - loss: 0.9688 - categorical_accuracy: 0.1841
Epoch 6/60
126/126 [==============================] - 6s 17ms/step - loss: 0.7668 - categorical_accuracy: 0.2071
Epoch 7/60
126/126 [==============================] - 5s 16ms/step - loss: 0.5460 - categorical_accuracy: 0.2230
Epoch 8/60
126/126 [==============================] - 5s 17ms/step - loss: 0.4889 - categorical_accuracy: 0.2373
Epoch 9/60
126/126 [==============================] - 5s 17ms/step - loss: 0.4597 - categorical

In [16]:
loss, accuracy = model.evaluate(decoded_test)

16/16 [==============================] - 3s 23ms/step - loss: 0.4184 - categorical_accuracy: 0.2281


In [17]:
for x_batch, y_batch in decoded_test.take(1):
    print(tf.keras.activations.sigmoid(model(x_batch)))

tf.Tensor(
[[2.97253668e-01 1.40730351e-01 5.03302105e-02 3.40484381e-01
  8.40265527e-02 1.71332508e-01 1.01809308e-01 1.17715262e-01
  1.07253447e-01 7.04891235e-02 3.40034701e-02 9.29218680e-02]
 [1.77643299e-01 3.91686171e-01 1.11929864e-01 1.98236644e-01
  1.67848080e-01 1.88575789e-01 1.33046005e-02 1.24368900e-02
  4.61958498e-02 7.21370503e-02 7.52775522e-04 1.17185777e-02]
 [3.25750709e-01 1.75102606e-01 9.97960716e-02 3.13333869e-01
  1.01970620e-01 2.11723059e-01 6.33794591e-02 5.66635616e-02
  9.70811993e-02 4.04447541e-02 5.28191477e-02 9.98495296e-02]
 [3.13887209e-01 1.75368920e-01 8.35188627e-02 3.07842821e-01
  1.05433330e-01 1.42121717e-01 1.49081632e-01 1.89094037e-01
  2.03690529e-01 1.26142889e-01 8.23792219e-02 1.23838164e-01]
 [3.57293636e-01 1.51078448e-01 6.57494664e-02 3.20014268e-01
  8.25272277e-02 1.36222288e-01 1.30962461e-01 1.03817508e-01
  1.16383888e-01 7.37531036e-02 1.00800648e-01 1.26166567e-01]
 [2.51650840e-01 1.64203569e-01 5.53012639e-02 2.45685